In [1]:
# Assume a collection of playlists, P (which they refer to as the Case Base).

# User inputs to the algorithm:
# - seed song (s)
# - desired length (N)
# Hyperparameter (k)

import pandas as pd
import os, sys, time, random
import json
import numpy as np
import keras
import sqlite3
import math
import functools
import operator
import asyncio

k = 20
# seed = 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak' # Britney Spears - Toxic
seed = 'spotify:track:4aVuWgvD0X63hcOCnZtNFA' # Toto - Hold the Line
conn = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
cur = conn.cursor()

2023-03-23 00:57:56.256114: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 00:57:56.389531: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-23 00:57:56.393290: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-23 00:57:56.393314: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
def var_attr(l, gamma, delta):
    mul_attr = 1
    i = 0
    while i < len(l) - 1:
        j = i + 1
        while j < len(l) and j - i < gamma:
            if l[j] != 0 and l[i] != 0 and abs(l[j] - l[i]) > delta:
                mul_attr *= ((j-i) / gamma)
                break

        i += 1

    return mul_attr

In [3]:
def var(p): # todo: fine-tuning deltas and gammas
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    delta_diffs = [0.25, 0.25, 0.25, 0.25, 0.25]
    gammas = [3, 3, 3, 3, 3]
    feature_list = np.array(c.execute(f"SELECT tf.acousticness, tf.instrumentalness, tf.speechiness, tf.energy, tf.valence FROM track_features as tf JOIN playlist_tracks as pt WHERE tf.track_uri = pt.track_uri AND pt.pid = '{p}' ORDER BY pt.pindex ASC;").fetchall()).transpose()
    total_attrs = [var_attr(feature_list[i], gammas[i], delta_diffs[i]) for i in range(len(gammas))]
    total = 1
    for attr in total_attrs:
        total *= attr

    return total

In [4]:
def rel(q, t, checked=True):
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    if not checked:
        if t in q:
            phi = int(c.execute(f"SELECT COUNT(*) FROM seq2_simple WHERE track_uri1 = '{q[0]}' AND track_uri2 = '{q[1]}';").fetchone()[0])
        else:
            return -1 # t not in q error
    else:
        phi = int(c.execute(f"SELECT COUNT(*) FROM seq2_simple WHERE track_uri1 = '{q[0]}' AND track_uri2 = '{q[1]}';").fetchone()[0])
    
    if False: # future variability in pattern size, adding weighting based on length
        alpha = 0.5
        theta = 2
        lq = len(q)
    
    beta = 0.5 # popularity weighting
    psi = 0
    x = q[0]
    if q[0] == t:
        x = q[1]

    psi = pow(int(c.execute(f"SELECT COUNT(*) FROM playlist_tracks WHERE track_uri = '{x}';").fetchone()[0]), beta)
    rel = phi / psi

    return rel

In [5]:
''' for seq2_occurences not seq2_simple '''
# def rel(q, t, checked=True):
#     if not checked:
#         if t in q:
#             phi = int(cur.execute(f"SELECT prevalence FROM seq2_occurences WHERE track_uri1 = '{q[0]}' AND track_uri2 = '{q[1]}';").fetchone()[0])
#         else:
#             return -1 # t not in q error
#     else:
#         phi = int(cur.execute(f"SELECT prevalence FROM seq2_occurences WHERE track_uri1 = '{q[0]}' AND track_uri2 = '{q[1]}';").fetchone()[0])
    
#     if False: # future variability in pattern size, adding weighting based on length
#         alpha = 0.5
#         theta = 2
#         lq = len(q)
    
#     beta = 0.5 # popularity weighting
#     psi = 0
#     x = q[0]
#     if q[0] == t:
#         x = q[1]

#     psi = pow(int(cur.execute(f"SELECT COUNT(*) FROM playlist_tracks WHERE track_uri = '{x}';").fetchone()[0]), beta)
#     rel = phi / psi

#     return rel

' for seq2_occurences not seq2_simple '

In [6]:
# containing = cur.execute(f"SELECT pid, pindex FROM playlist_tracks WHERE track_uri = '{seed}';").fetchall()
# totals = []
# for c in containing:
#     rel_prev = 0
#     rel_after = 0
#     prev = cur.execute(f"SELECT track_uri FROM playlist_tracks WHERE pid = '{c[0]}' AND pindex+1 = '{c[1]}';").fetchone()
#     after = cur.execute(f"SELECT track_uri FROM playlist_tracks WHERE pid = '{c[0]}' AND pindex-1 = '{c[1]}';").fetchone()
#     occurences = 0
#     if prev is not None:
#         rel_prev = rel([prev[0], seed], seed)
#     if after is not None:
#         rel_after = rel([seed, after[0]], seed)

#     coherence = rel_prev + rel_after
#     variety = var(c[0])
    
#     rho = variety * coherence
#     totals.append([c, rho])
    
# containing_sorted = sorted(totals, key = lambda prevalence: prevalence[1], reverse=True)

# best_k_playlists = [p[0] for p in containing_sorted[:k]] # list of k sorted playlist pids
# best_k_playlists

In [7]:
from multiprocessing import Pool
from multiprocessing import Process
def test(x):
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    return c.execute(f"SELECT * FROM playlist_tracks LIMIT {x}, 100;").fetchall()[0]

with Pool(5) as p:
        print(p.map(test, [0, 100, 200, 300]))


[(0, 0, 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI'), (2, 9, 'spotify:track:7tIJDktakabGoHjwTTa35W'), (3, 45, 'spotify:track:0tOyrixMQ17NUznPIxYtVD'), (5, 2, 'spotify:track:1V4jC0vJ5525lEF1bFgPX2')]


In [8]:
containing = cur.execute(f"SELECT pid, pindex FROM playlist_tracks WHERE track_uri = '{seed}';").fetchall()
totals = []

prev_cases = [f"(pid = {c[0]} AND pindex-1 = {c[1]})" for c in containing]
prev_split = []
for x in range(0, len(prev_cases), 900):
    prev_split.append("OR".join(prev_cases[x:x+900]))

after_cases = [f"(pid = {c[0]} AND pindex+1 = {c[1]})" for c in containing]
after_split = []
for x in range(0, len(after_cases), 900):
    after_split.append("OR".join(after_cases[x:x+900]))

def fetch(x):
    con = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
    c = con.cursor()
    return c.execute(f"SELECT pid, track_uri FROM playlist_tracks WHERE {x} ORDER BY pid;").fetchall()

In [9]:
print(len(prev_split))
print(len(after_split))

6
6


In [10]:
fetched_prev_tracks = []
fetched_after_tracks = []
# with Pool() as p:
# with Pool((max(len(prev_cases), len(after_cases)) // 900)*2) as p:
with Pool(len(prev_split) + len(after_split)) as p:
    fetched_prev_tracks = p.map_async(fetch, prev_split)
    fetched_after_tracks = p.map_async(fetch, after_split)

    p.close()
    p.join()

# fetched_after_tracks.wait()
# fetched_prev_tracks.wait()


In [11]:
fetched_prev_tracks = [item for sublist in fetched_prev_tracks.get() for item in sublist]
fetched_after_tracks = [item for sublist in fetched_after_tracks.get() for item in sublist]

fetched_prev_tracks = dict(fetched_prev_tracks)
fetched_after_tracks = dict(fetched_after_tracks)

In [12]:
def rpp(i):
# for i in fetched_prev_tracks.keys() & fetched_after_tracks.keys():
    # rel_prev = Process(target=rel, args=([fetched_prev_tracks[i], seed], seed))
    # rel_prev.run()

    # rel_after = Process(target=rel, args=([seed, fetched_after_tracks[i]], seed))
    # rel_after.run()
    rel_prev = rel([fetched_prev_tracks[i], seed], seed)
    rel_after = rel([seed, fetched_after_tracks[i]], seed)
    # with Pool(2) as q:
    #     rel_prev, rel_after = q.map(rel, [([fetched_prev_tracks[i], seed], seed),([seed, fetched_after_tracks[i]], seed)])

    coherence = rel_prev + rel_after
    variety = var(i)
    
    rho = variety * coherence
    # totals.append([i, rho])
    return [i, rho]

In [13]:
def rpp_before(i):
# for i in fetched_prev_tracks.keys() - fetched_after_tracks.keys():
    rel_prev = rel([fetched_prev_tracks[i], seed], seed)

    variety = var(i)
    
    rho = variety * rel_prev
    # totals.append([i, rho])
    return [i, rho]

In [14]:
def rpp_after(i):
# for i in fetched_after_tracks.keys() - fetched_prev_tracks.keys():
    rel_after = rel([fetched_prev_tracks[i], seed], seed)

    variety = var(i)
    
    rho = variety * rel_after
    # totals.append([i, rho])
    return [i, rho]

In [15]:
# conn.close()
# conn = sqlite3.connect("/mnt/z/other/spotify_backup.db", check_same_thread=False)
# cur = conn.cursor()
# c = cur.execute("SELECT * FROM tracks LIMIT 20;").fetchall()
# print(c)

In [16]:
intersection = fetched_prev_tracks.keys() & fetched_after_tracks.keys()
# with Pool(len(set(fetched_prev_tracks.keys()).union(set(fetched_after_tracks.keys())))) as p:
with Pool(16) as p:
    t1 = p.map_async(rpp, intersection)
    t2 = p.map_async(rpp_before, fetched_prev_tracks.keys() - fetched_after_tracks.keys())
    t3 = p.map_async(rpp_after, fetched_after_tracks.keys() - fetched_prev_tracks.keys())

    p.close()
    p.join()

# with Pool(len(fetched_prev_tracks.keys()) - len(intersection)) as p:
# with Pool(10) as p2:
# with Pool(len(fetched_prev_tracks.keys()) - len(intersection)) as p:
# with Pool(10) as p3:
totals = t1.get() + t2.get() + t3.get()
containing_sorted = sorted(totals, key = lambda prevalence: prevalence[1], reverse=True)

best_k_playlists = [p[0] for p in containing_sorted[:k]] # list of k sorted playlist pids
best_k_playlists

KeyboardInterrupt: 

In [ ]:

best_k_playlists_string = ", ".join([f"'{b}'" for b in best_k_playlists])
candidate_songs = [s[0] for s in cur.execute(f"SELECT DISTINCT track_uri FROM playlist_tracks WHERE pid in ({best_k_playlists_string});").fetchall()]
current_playlist = [seed]
candidate_playlists = [current_playlist]
candidate_songs

In [ ]:
def check_pair(x, y): # check whether sequence <x, y> occurs in any of the k retrieved playlists and count them
    occurences = 0
    containing_playlists = [c for c in cur.execute(f"SELECT p1.pid, p1.pindex, p2.index FROM playlist_tracks as p1 JOIN playlist_tracks as p2 WHERE p1.track_uri = {x} AND p2.track_uri = {y} AND p1.pid = p2.pid AND p1.pid in ({best_k_playlists_string}));")]
    for playlist in containing_playlists:
        if playlist[1] == playlist[2] - 1:
            # return 1
            occurences += 1
            
    return occurences

In [ ]:
def h(T_prime, seq, u):
    r = rel(seq, u) # relevance of pattern seq to added song u
    return r * var(T_prime)

    # todo: look-ahead factor L

In [ ]:
N = 8
while len(current_playlist) < N:
    t = current_playlist[0]
    T = current_playlist[-1]
    successors_of_current_playlist = []
    for u in candidate_songs:
        pre_occurences = check_pair(u, t)
        if pre_occurences > 0:
            successor_playlist = [u] + current_playlist
            successors_of_current_playlist.append([pre_occurences, successor_playlist, [u,current_playlist[0]]])
        post_occurences = check_pair(T, u)
        if post_occurences > 0:
            successor_playlist = current_playlist + [u]
            successors_of_current_playlist.append([post_occurences, successor_playlist, [current_playlist[-1],u]])
    if len(successors_of_current_playlist) + len(successors_of_current_playlist) == 0:
        # discard current_playlist
        print("failed to create playlist")
    else: # pop new current_playlist from candidate_playlists
    	# sort successors_of_current_playlist using some measure of quality 
        for x in range(len(successors_of_current_playlist)): # todo: more efficient var check?
            successors_of_current_playlist[x][3] = h(successors_of_current_playlist[x][1], successors_of_current_playlist[x][2], u)
        
        successors_of_current_playlist = sorted(successors_of_current_playlist, key=lambda x: x[3], reverse=True)
        print(successors_of_current_playlist)
        current_playlist = successors_of_current_playlist[0][1]
        saved_playlists = successors_of_current_playlist[1::]
        # insert the saved_playlists into candidate_playlists
        candidate_playlists += saved_playlists
        # In fact, they should be inserted in such a way as to keep candidate_playlists 
        # ordered also by their quality, but ignore this for now

# If we reach here, we exited the loop, i.e. we have a playlist of length L
print(current_playlist)
